# Lab 06. VGG11 모델과  ResNet18 모델 앙상블 실습
---

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transform

from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.models import vgg11, resnet18

from sklearn.metrics import accuracy_score

In [2]:
# GPU 설정 (사용 가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("device >>" , device)

device >> cuda


In [3]:
# 데이터셋 불러오기 .
train_transform = transform.Compose([
    transform.RandomHorizontalFlip(),
    transform.RandomVerticalFlip(),
    transform.RandAugment(),
    transform.ToTensor(),
    transform.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))    # 이미지를 -1 ~ 1로 정규화
])

test_transform = transform.Compose([
    transform.ToTensor(),
    transform.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))     # 이미지를 -1 ~ 1로 정규화
])

In [4]:
# 데이터 정의
train_dataset = CIFAR10(root="./data", train=True, download=True, transform=train_transform)
test_dataset = CIFAR10(root="./data", train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=246, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=246, shuffle=False, num_workers=2, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# vgg11 모델 과 resnet18 모델 정의
vgg_model = vgg11(pretrained=True)
resnet_model = resnet18(pretrained=True)

num_features_vgg = vgg_model.classifier[6].in_features
num_features_resnet = resnet_model.fc.in_features

vgg_model.classifier[6] = nn.Linear(num_features_vgg, 10)
resnet_model.fc = nn.Linear(num_features_resnet, 10)

c:\Miniconda\envs\MS_AI_SCHOOL\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Miniconda\envs\MS_AI_SCHOOL\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Miniconda\envs\MS_AI_SCHOOL\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the

In [6]:
# Voting 앙상블 모델 정의
class EnsembleModel(nn.Module) :
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x):
        outputs = [model(x) for model in self.models]
        outputs = torch.stack(outputs, dim=0)
        avg_output = torch.mean(outputs, dim=0)

        return avg_output

ensemble_model = EnsembleModel([vgg_model, resnet_model])
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(ensemble_model.parameters(), lr=0.001)

In [7]:
# 모델 학습 함수 정의
def train(model, device, train_loader , optimizer, criterion) :
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader) :
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, device, test_loader) :
    model.eval()
    predictions = []
    targets = []

    with torch.no_grad() :
        for data, target in test_loader :
            data, target = data.to(device) , target.to(device)
            output = model(data)
            _, predicted = torch.max(output,1)
            predictions.extend(predicted.cpu().numpy())
            targets.extend(target.cpu().numpy())

    acc = accuracy_score(targets, predictions)
    return acc

In [8]:
# 앙상블된 모델 예측 합치는 함수 정의
def combine_predictions(predictions) :
    combined = torch.cat(predictions, dim=0)
    _, predicted_lables = torch.max(combined, 1)
    return predicted_lables

if __name__ == '__main__' :
    for epoch in range(1, 20) :
        print(f"Train model ... {epoch}")
        ensemble_model = ensemble_model.to(device)
        train(ensemble_model, device, train_loader, optimizer, criterion)
        predictions = []
        with torch.no_grad() :
            for data, _ in test_loader  :
                data = data.to(device)
                output = ensemble_model(data)
                predictions.append(output)

        combine_predictions_ = combine_predictions(predictions)
        acc = accuracy_score(test_dataset.targets, combine_predictions_.cpu().numpy())

        print(f"epoch >> [{epoch}] acc >> [{acc:.2f}]")

Train model ... 1
epoch >> [1] acc >> [0.68]
Train model ... 2
epoch >> [2] acc >> [0.72]
Train model ... 3
epoch >> [3] acc >> [0.74]
Train model ... 4
epoch >> [4] acc >> [0.74]
Train model ... 5
epoch >> [5] acc >> [0.76]
Train model ... 6
epoch >> [6] acc >> [0.76]
Train model ... 7
epoch >> [7] acc >> [0.77]
Train model ... 8
epoch >> [8] acc >> [0.77]
Train model ... 9
epoch >> [9] acc >> [0.78]
Train model ... 10
epoch >> [10] acc >> [0.79]
Train model ... 11
epoch >> [11] acc >> [0.78]
Train model ... 12
epoch >> [12] acc >> [0.78]
Train model ... 13
epoch >> [13] acc >> [0.78]
Train model ... 14
epoch >> [14] acc >> [0.79]
Train model ... 15
epoch >> [15] acc >> [0.79]
Train model ... 16
epoch >> [16] acc >> [0.79]
Train model ... 17
epoch >> [17] acc >> [0.80]
Train model ... 18
epoch >> [18] acc >> [0.80]
Train model ... 19
epoch >> [19] acc >> [0.81]
